# Creating OME-ZARR images for CellDIVE

In [2]:
from pydantic_zarr.v2 import ArraySpec
from numcodecs import Zstd
import zarr
from pathlib import Path
import tempfile
#from data_helpers import get_directory_contents
from ome_zarr_models.v04 import Image
from ome_zarr_models.v04.axes import Axis
import math
import scipy.ndimage

In [ ]:
array_spec = ArraySpec(
    shape=heart_image.shape,
    dtype=heart_image.dtype,
    chunks=(32, 32, 32),
    compressor=Zstd(level=5),
)
print(array_spec)

In [ ]:
store = zarr.MemoryStore()
zarr_array = array_spec.to_zarr(store=store, path="/")
zarr_array

Optional temp storage approach

In [ ]:
temp_dir = tempfile.TemporaryDirectory()
temp_path = Path(temp_dir.name)
print(f"Created temporary directory at {temp_path}")
print(f"Directory contents before saving: {get_directory_contents(temp_path)}")

store = zarr.DirectoryStore(temp_dir.name)
zarr_array = array_spec.to_zarr(store=store, path="/")
print(f"Directory contents after creating array: {get_directory_contents(temp_path)}")

zarr_array[:] = heart_image[:]
print(f"Directory contents after adding data: {get_directory_contents(temp_path)}")

Creating OME-ZARR files

In [ ]:
voxel_size = 19.89
ome_zarr_image = Image.new(
    array_specs = [ArraySpec.from_zarr(zarr_array)],
    paths = ["level0"],
    axes = [
        Axis(name="x", type="space", unit="um"),
        Axis(name="y", type="space", unit="um"),
        Axis(name="z", type="space", unit="um")
    ],
    global_scale = [voxel_size, voxel_size, voxel_size],
    scales = [[1, 1, 1]],
    translations = [[0, 0, 0]],
    name = "heart_image"
)
print(ome_zarr_image)

In [ ]:
ome_store = zarr.MemoryStore()
ome_group = ome_zarr_image.to_zarr(ome_store, path='')
print(ome_group)

In [ ]:
level0_array = ome_group[ome_zarr_image.attributes.multiscales[0].datasets[0].path]

In [ ]:
print("Before filling:")
print(level0_array[:, :, 0])
# Fill the data
level0_array[:] = zarr_array[:]
print("After filling:")
print(level0_array[:, :, 0])

Adding down sampled levels

In [ ]:
full_res_spec = ArraySpec.from_array(zarr_array)
print("Original array specification: ", full_res_spec)

downsample_levels = [0, 1, 2]
downsampled_specs = [
    full_res_spec.model_copy(
        update={"shape": tuple(math.ceil(i / 2**d) for i in full_res_spec.shape)
    }) for d in downsample_levels
]
print("Downsampled array specifications: ", downsampled_specs)

In [ ]:
multiscale_image = Image.new(
    array_specs = downsampled_specs,
    paths = [f"level{d}" for d in downsample_levels],
    axes = [
        Axis(name="x", type="space", unit="um"),
        Axis(name="y", type="space", unit="um"),
        Axis(name="z", type="space", unit="um")
    ],
    global_scale = [voxel_size, voxel_size, voxel_size],
    scales = [[2**d, 2**d, 2**d] for d in downsample_levels],
    translations = [[0, 0, 0] for d in downsample_levels],
    name = "heart_image"
)
print(multiscale_image)

In [ ]:
multiscale_store = zarr.MemoryStore()
multiscale_group = multiscale_image.to_zarr(multiscale_store, path='')
print(multiscale_group)

In [ ]:
multiscale_group['level0'][:] = zarr_array[:]

for dataset in multiscale_image.datasets[0][1:]:
    scale = dataset.coordinateTransformations[0].scale
    print(scale)
    downsampled_array = scipy.ndimage.zoom(zarr_array[:], zoom=[1 / s for s in scale])
    # Write downsampled arrays to Zarr storage
    multiscale_group[dataset.path][:] = downsampled_array